In [2]:
import face_rec

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [3]:
import pandas as pd
import numpy as np
import cv2

In [4]:
face_rec.r.ping()

True

#### Step:1 : Extract Data fromm Database

In [5]:
# 'academy:register'
name = 'academy:register'
retrieve_dict = face_rec.r.hgetall(name)
retrieve_series = pd.Series(retrieve_dict)
retrieve_series = retrieve_series.apply(lambda x : np.frombuffer(x,dtype=np.float32))
index = retrieve_series.index
index = list(map(lambda x : x.decode(),index))
retrieve_series.index = index
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ['name_role','facial_features']
retrieve_df[['Name','Role']] = retrieve_df['name_role'].apply(lambda x : x.split('@')).apply(pd.Series)
retrieve_df

,name_role,facial_features,Name,Role
0,muskan@Teacher,"[0.73370165, -0.46797958, 1.2609189, 1.6116163...",muskan,Teacher
1,Angelina Jolie@Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291...",Angelina Jolie,Student
2,Chris Evans@Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073...",Chris Evans,Student
3,Morgan Freeman@Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788...",Morgan Freeman,Teacher
4,Scarlett Johansson@Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967...",Scarlett Johansson,Student
5,Barack Obama@Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965...",Barack Obama,Teacher
6,Akshat@Student,"[-0.9122446, 0.31055236, 0.030251471, 0.167943...",Akshat,Student
7,Aviral@Student,"[-0.060164735, -0.20658961, -0.95173484, -0.99...",Aviral,Student


### Step : 2 Get real-time prediction

In [6]:
cap = cv2.VideoCapture(0) # default 0-1 for external
while True:
    ret, frame = cap.read()
    if ret == False:
        break
    pred_frame = face_rec.face_prediction(frame, retrieve_df, 'facial_features', ['Name','Role'], thresh=0.5)
    #cv2.imshow('frame',frame)
    cv2.imshow('prediction',pred_frame)


    if cv2.waitKey(1) == 27: # press Esc for closing the capture window
        break
cv2.destroyAllWindows()
cap.release()